In [1]:
import pandas as pd

In [3]:
file_path ='itmo_sber.csv'
data = pd.read_csv('itmo_sber.csv')

In [4]:
print(data.isnull().sum())

Unnamed: 0                    0
Задача                        0
Обстановка                    0
Оптимальный план              0
Задача en                     0
Обстановка en                 0
Оптимальный план en           0
Предсказанный план            0
Успех предсказанного плана    0
dtype: int64


In [5]:
print(data.head())

   Unnamed: 0                          Задача  \
0           0   Приготовить горячий шоколад.    
1           1     Заказать доставку еды домой   
2           2  Реставрация старого комода\r\n   
3           3            Приготовить сырники.   
4           4               Нарезать колбасу.   

                                          Обстановка  \
0  кухня, молоко, какао, сахар, кастрюля, ложка, ...   
1  мобильный телефон, ресторанное меню, интернет,...   
2  Старый комод, повреждения, шпатель, шкурка, ма...   
3  Кухня, столешница, творог, мука, яйцо, сахар, ...   
4  Кухня, колбаса, нож, доска для резки, холодиль...   

                                    Оптимальный план  \
0  Откройте дверь. Зайдите на кухню. Закрйоте две...   
1  Найдите стол. Подойдите к столу. Найдите мобил...   
2  Подойдите к комоду.Найдите шкурку.Возьмите шку...   
3  Подойти к столешнице. Найти творог. Взять твор...   
4  Подойти к холодильнику. Открыть холодильник. Н...   

                               

In [6]:
print(data['Успех предсказанного плана'].value_counts())

Успех предсказанного плана
1    3234
0    1600
Name: count, dtype: int64


In [7]:
print(data.describe())

        Unnamed: 0  Успех предсказанного плана
count  4834.000000                 4834.000000
mean   2416.500000                    0.669011
std    1395.599931                    0.470618
min       0.000000                    0.000000
25%    1208.250000                    0.000000
50%    2416.500000                    1.000000
75%    3624.750000                    1.000000
max    4833.000000                    1.000000


In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['Задача'])

In [10]:
print(X.shape)

(4834, 5402)


In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('russian'))

def preprocess_text(text):
    words = nltk.word_tokenize(text)
    words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words and word.isalpha()]
    return ' '.join(words)

data['Задача'] = data['Задача'].apply(preprocess_text)
data['Обстановка'] = data['Обстановка'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
# Инициализация векторизатора
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Объединяем текстовые данные для векторизации
combined_text = data['Задача'] + " " + data['Обстановка']
X = tfidf_vectorizer.fit_transform(combined_text)

# Целевая переменная
y = data['Успех предсказанного плана']

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# Объединение текстовых полей
data['combined_text'] = data['Задача en'] + ' ' + data['Обстановка en'] + ' ' + data['Предсказанный план']

# Функция для очистки и нормализации текста
def clean_text(text):
    text = text.lower()  # Приведение к нижнему регистру
    text = re.sub(r'\s+', ' ', text)  # Удаление лишних пробелов
    text = re.sub(r'[^\w\s]', '', text)  # Удаление специальных символов
    return text

# Применение функции очистки к каждой строке объединенного текстового поля
data['cleaned_text'] = data['combined_text'].apply(clean_text)

# Подготовка данных и целевой переменной
X = data['cleaned_text']
y = data['Успех предсказанного плана']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Векторизация текста с использованием TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # Ограничение на количество признаков для упрощения модели
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

X_train_tfidf.shape, X_test_tfidf.shape


((3867, 1000), (967, 1000))

In [14]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Загрузка предобученной модели BERT и токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

# Предварительная обработка текста
def preprocess_text(text):
    text = text.lower()
    text = text.strip()
    return text

data['Предсказанный план'] = data['Предсказанный план'].apply(preprocess_text)
data['Оптимальный план'] = data['Оптимальный план'].apply(preprocess_text)

# Функция для получения эмбеддингов текста с помощью BERT
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
    return embeddings

# Получение эмбеддингов для "Предсказанного плана" и "Оптимального плана"
predicted_plan_embeddings = np.array([get_bert_embeddings(text) for text in data['Предсказанный план']])
optimal_plan_embeddings = np.array([get_bert_embeddings(text) for text in data['Оптимальный план']])

# Вычисление сходства текстов с помощью косинусного сходства
cosine_similarities = [np.dot(pred, opt.T) / (np.linalg.norm(pred) * np.linalg.norm(opt)) for pred, opt in zip(predicted_plan_embeddings, optimal_plan_embeddings)]

# Подготовка данных и целевой переменной для классификации
X = np.array(cosine_similarities).reshape(-1, 1)
y = data['Успех предсказанного плана']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


C:\Users\kiril\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kiril\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Accuracy of the model: 0.6608066184074457


In [15]:
# Обучение классификатора
model = LogisticRegression()
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of the model: {accuracy}')

Accuracy of the model: 0.6608066184074457
